<a href="https://colab.research.google.com/github/majidsadrii/self-supervised/blob/main/self_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. *Imports*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

import matplotlib.pyplot as plt


import numpy as np 
import pandas as pd
from PIL import Image
import os
import os.path




from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

from __future__ import absolute_import
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
import math

! pip install colorama
from colorama import Fore

cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
% cd /content/drive/MyDrive/my_code/25_pso_second_step/cond_with_rotation
# % cd /content/drive/MyDrive/cond_with_rotation

/content/drive/MyDrive/my_code/25_pso_second_step/cond_with_rotation


In [8]:
from dataloader.dataloader import CIFAR10_modified , CIFAR100_modified
from visulization.dataset_vis import rotated_dataset_plot
from model.dense import CondenseNet

# 2. Parameters

In [ ]:
reslut_name = 'condens_with_selfsupervised6'

##################
# training parameter
batch_size = 32
weight_pr = 1

learning_rate = 0.1
momentum = 0.9
W_decay  = 0.001
lr_type = 'cosine'
##################


unsupervised_training =True
main_dataset  = 'Cifar10'
pre_text_dataset = 'Cifar10'

##############################
# CondenseNet parameters
model_parm ={}
model_parm['stages']  =  [14,14,14]
model_parm['data']  = 'cifar10'
model_parm['growth'] =[8,16,32]
model_parm['num_classes'] = 10
model_parm['group_1x1'] = 4
model_parm['group_3x3'] = 4
model_parm['bottleneck'] = 4
model_parm['condense_factor'] =4
model_parm['dropout_rate'] = 0
###############################

##################### saving parameters as a csv file

info = model_parm
info['batch_size'] = batch_size
info['weight_pr '] = weight_pr 
info['learning_rate']  = learning_rate 
info['momentum']  =  momentum
info['W_decay']  =  W_decay

df = pd.DataFrame(info)
df_name = '{}.csv'.format(reslut_name)
df.to_csv(df_name)

In [9]:
! ls

 best_condens_without_ss        condens_with_selfsupervised2.pt
 best_condens_without_ss2       condens_with_selfsupervised3
 best_condens_without_ss3       condens_with_selfsupervised3.pt
 best_condens_without_ss4       condens_with_selfsupervised4
 best_condens_without_ss5       condens_with_selfsupervised4.pt
 ch_condens_without_ss2.pt      condens_with_selfsupervised5
 ch_condens_without_ss3.pt      condens_with_selfsupervised5.pt
 ch_condens_without_ss4.pt      condens_with_selfsupervised.csv
 ch_condens_without_ss5.pt      condens_with_selfsupervised.png
 ch_condens_without_ss.pt       condens_with_selfsupervised.pt
 condens_without_ss2.csv        cond_model_without_selfsupervised.ipynb
 condens_without_ss2.png       'Copy of cond2.ipynb'
 condens_without_ss3.csv        data
 condens_without_ss3.png        dataloader
 condens_without_ss4.csv        main.ipynb
 condens_without_ss4.png        model
 condens_without_ss5.csv       'result '
 condens_without_ss5.png        runs2022072

In [ ]:
transform = transforms.Compose(    [transforms.ToTensor(),transforms.Normalize(mean=[0.4914, 0.4824, 0.4467], std=[0.2471, 0.2435, 0.2616])])
if main_dataset  == 'Cifar10':
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False)

  classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
if unsupervised_training:
  if pre_text_dataset == 'Cifar100':
    pretext_data = CIFAR100_modified(root='./data', train=True,download=True, transform=transform)
    pretext_data_loader = torch.utils.data.DataLoader(pretext_data, batch_size=batch_size, shuffle=True)
  elif  pre_text_dataset == 'Cifar10':
    pretext_data = CIFAR10_modified(root='./data', train=True,download=True, transform=transform)
    pretext_data_loader = torch.utils.data.DataLoader(pretext_data, batch_size=batch_size, shuffle=True)
# plotting some sample
  rotated_dataset_plot(pretext_data_loader)

# 3.Model

In [ ]:
my_model = CondenseNet( model_parm['stages'] ,model_parm['data'], model_parm['growth'],model_parm['num_classes'],model_parm['group_1x1'],model_parm['group_3x3'],model_parm['bottleneck'],model_parm['condense_factor'],model_parm['dropout_rate'] )


In [ ]:
my_model.to(device)

# 4. Model training

In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(my_model.parameters(), lr=0.0001)
# optimizer = optim.SGD(my_model.parameters(), lr=learning_rate, momentum=moment,weight_decay=W_decay)
optimizer = torch.optim.SGD(my_model.parameters(), lr=learning_rate,
                                momentum=momentum,
                                weight_decay=W_decay,
                                nesterov=True)

In [ ]:
def adjust_learning_rate(optimizer, epoch, EPOCHS,learning_rate ,batch=None,
                         nBatch=None, method='cosine'):
    if method == 'cosine':
        T_total = EPOCHS  * nBatch
        T_cur = (epoch % EPOCHS ) * nBatch + batch
        lr = 0.5 * learning_rate * (1 + math.cos(math.pi * T_cur / T_total))
    elif method == 'multistep':
       
            lr, decay_rate = learning_rate, 0.1
            if epoch >= EPOCHS * 0.75:
                lr *= decay_rate**2

            elif epoch >= EPOCHS * 0.5:
                lr *= decay_rate

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [ ]:
def train_one_epoch(epoch, tb_writer, total_bach,lr_type,optimizer,EPOCHS,learning_rate ):

    running_loss_1 = 0.
    last_loss_1 = 0.
    running_loss_2 = 0.
    last_loss_2 = 0.
    acc = 0
    tot = 0
    cor = 0
    acc2 = 0
    tot2 = 0
    cor2 = 0
    training_loss = 0
    running_lr = None
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(trainloader,0):

        lr = adjust_learning_rate(optimizer, epoch,EPOCHS = EPOCHS,learning_rate = learning_rate, batch=i,
                                  nBatch=total_bach, method=lr_type )
        if running_lr is None:
            running_lr = lr


      # Zero your gradients for every batch!
        optimizer.zero_grad()     
        image,label = data
        image,label = image.to(device),label.to(device)
        # Make predictions for this batch
        outputs,_ = my_model(image)            
        loss_1 = criterion(outputs,label) 
        running_loss_1 += loss_1.item()        
        _, pr= torch.max( outputs.data, 1)
 
        # _, real= torch.max(label.data, 1)

        tot += label.size(0)
        cor += (pr ==  label).sum().item()
        acc = 100 * cor / tot    
        
        if unsupervised_training:
          _,image_r,rotation = next(iter(pretext_data_loader))            
          image_r,rotation =  image_r.to(device),rotation.to(device)      
          _,outputs_r = my_model( image_r) 
          loss_2 = criterion(outputs_r,rotation)
          running_loss_2 += loss_2.item()
          loss =  loss_1 + weight_pr *loss_2            
          _, pr2= torch.max(outputs_r.data, 1)
          _, real2= torch.max(rotation.data, 1)
          tot2 += rotation.size(0)
          cor2 += (pr2 == real2).sum().item()
          acc2 = 100 * cor2 / tot2
        else:
          loss =  loss_1

        loss.backward()      
        optimizer.step()      

        last_loss_1 += loss_1 
        last_loss_2 += loss_2
        if i % 50 == 49:

          R1 = running_loss_1/50
          R2 =running_loss_2/50
          print(Fore.BLUE,'{}/ {} train_l1: {} train_l2: {}, accu is {}  pretex_accu is {} time is {}'.format(i + 1,total_bach, '%.5f' %(R1),'%.5f' %(R2),'%.4f' %acc,'%.4f' %acc2,'%.5f' %(time.time() - start_time)))
          
          running_loss_1 = 0.
          running_loss_2 = 0. 

        
    PATH ='{}.pt'.format(reslut_name)
    
    torch.save({
                'epoch': epoch,
                'model_state_dict': my_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, PATH)

    return (last_loss_1/total_bach),(last_loss_2/total_bach),acc,acc2,running_lr

In [ ]:
def plot_result(train_loss_main,test_loss_main,train_accuracy_main,test_accuracy_main):
    E = [i for i in range(len(train_loss_main))]

    fig, (axs, ax2,ax3) = plt.subplots(3)
    t_train_loss_main = [train_loss_main[i].cpu().detach().numpy()  for i  in range(len(train_loss_main))]
    s_test_loss_main =  [test_loss_main[i].cpu().detach().numpy()  for i  in range(len(test_loss_main))]
    axs.plot(E, t_train_loss_main)
    axs.plot(E,s_test_loss_main)    
    axs.set_xlabel('epoch')
    axs.set_ylabel('loss')
    axs.grid(True)

    t_train_accuracy_main  = [train_accuracy_main [i]  for i  in range(len(train_accuracy_main ))]
    s_test_accuracy_main =  [test_accuracy_main[i] for i  in range(len(test_accuracy_main))]
    ax2.plot(E, t_train_accuracy_main )
    ax2.plot(E,s_test_accuracy_main)    
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('accuracy')
    ax2.grid(True)

    t_train_accuracy_pre = [train_accuracy_pre[i]  for i  in range(len(train_loss_main))]
    ax3.plot(E, t_train_accuracy_pre )
    ax3.set_xlabel('epoch')
    ax3.set_ylabel('pretext_accuracy')

    fig.tight_layout()
    image_name =reslut_name +'.png'
    plt.savefig(image_name )

    dict = {'acc_train': t_train_accuracy_main, 
            'acc_test': s_test_accuracy_main,
            'loss_train': t_train_loss_main,
            'loss_test':s_test_loss_main}
    df = pd.DataFrame(dict)
    df_name = '{}.csv'.format(reslut_name)
    df.to_csv(df_name)

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
total_bach = len(trainloader)
total_batch_test = len(testloader)
writer = SummaryWriter('runs{}'.format(timestamp))
epoch_number = 0
import time
start_time = time.time()

EPOCHS = 200
best_vloss = 1_000_000.

train_loss_main =[]
train_accuracy_main =[]

train_loss_pre =[]
train_accuracy_pre =[]

test_loss_main =[]
test_accuracy_main =[]

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    my_model.train(True) 
   

    L1, L2,acc,acc2,lr= train_one_epoch(epoch_number, writer,total_bach,lr_type,optimizer,EPOCHS,learning_rate)
    train_loss_main.append(L1)    
    train_accuracy_main.append(acc)
    if unsupervised_training:
      train_loss_pre.append(L2)
      train_accuracy_pre.append(acc2)
 
    running_vloss = 0.0
    correct =0
    total = 0.
    my_model.train(False) 
    for i, vdata in enumerate(testloader ):
        with torch.no_grad():
          v_image, vlabels = vdata
          v_image, vlabels =  v_image.to(device), vlabels.to(device)
          voutputs,_  =  my_model(v_image)          
                      
          vloss = criterion(voutputs, vlabels)
          running_vloss += vloss

          _, predicted = torch.max(voutputs.data, 1)
          # _, real_label = torch.max(vlabels.data, 1)
          total += vlabels.size(0)
          correct += (predicted == vlabels.data).sum().item()
         
    accuracy = 100 * correct / total
    avg_vloss = running_vloss / (total_batch_test)
    test_loss_main.append(avg_vloss)
    test_accuracy_main.append(accuracy)


    print(Fore.GREEN,'for main taks,loss_test {},..... acc_test {}'.format(avg_vloss, accuracy))   
  

    if i %5 ==0:
          plot_result(train_loss_main,test_loss_main,train_accuracy_main,test_accuracy_main)

       # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = '{}'.format(reslut_name)
        torch.save(my_model.state_dict(),model_path)


  
    epoch_number += 1

 750/ 1563 train_l1: 0.33488 train_l2: 0.95210, accu is 88.1333  pretex_accu is 78.2375 time is 3141.16201
 800/ 1563 train_l1: 0.33739 train_l2: 0.96363, accu is 88.1289  pretex_accu is 78.1914 time is 3153.91006
 850/ 1563 train_l1: 0.39246 train_l2: 0.96715, accu is 88.0037  pretex_accu is 78.1397 time is 3166.58829
 900/ 1563 train_l1: 0.39316 train_l2: 0.95330, accu is 87.8958  pretex_accu is 78.1667 time is 3179.44159
 950/ 1563 train_l1: 0.36345 train_l2: 0.96528, accu is 87.8388  pretex_accu is 78.1283 time is 3192.22458
 1000/ 1563 train_l1: 0.37789 train_l2: 0.95380, accu is 87.7500  pretex_accu is 78.1594 time is 3205.02462
 1050/ 1563 train_l1: 0.37154 train_l2: 0.94549, accu is 87.6905  pretex_accu is 78.2083 time is 3217.90110
 1100/ 1563 train_l1: 0.34595 train_l2: 0.95879, accu is 87.7102  pretex_accu is 78.2244 time is 3230.64826
 1150/ 1563 train_l1: 0.34548 train_l2: 0.97098, accu is 87.7174  pretex_accu is 78.1658 time is 3243.64806
 1200/ 1563 train_l1: 0.35895 tra